In [1]:
import pandas as pd
import numpy as np

from memory_profiler import profile

from Pyfhel import Pyfhel, PyPtxt, PyCtxt

import torch
import torch.nn as nn

import time
import os
import sys

working_directory = "/home/falcetta/PINPOINT_Secret"

device = "cpu"
module_path = os.path.abspath(working_directory)
sys.path.append(module_path) 

from pycrcnn.net_builder.encoded_net_builder_ts import build_from_pytorch
from pycrcnn.crypto.crypto import encrypt_matrix, decrypt_matrix
from train_utils import *

from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
# HE = Pyfhel()    
# HE.contextGen(p=6514233, m=8192, intDigits=16, fracDigits=32) 
# HE.keyGen()
# HE.relinKeyGen(30, 3)

In [3]:
# a = 0.5

# print("First conv")
# a = (a * 0.1) - 0.3
# print(a)

# print("First square")
# a = a * a
# print(a)

# print("Second conv")
# a = (a * 0.2) - 0.1
# print(a)

# print("Second square")
# a = a * a
# print(a)

# print("First linear")
# a = (a * 0.1) + (a * 0.05) + (a * 0.3) - 0.1
# print(a)

# print("Second linear")
# a = (a * -0.2) + (a * 0.1) + (a * 0.1) - 0.4
# print(a)

# print("Third linear")
# a = (a * 0.3) + (a * -0.2) + (a * 0.3) + 0.2
# print(a)

In [4]:
# a = 0.51951235
# enc_a = HE.encryptFrac(a)

# print("First conv")
# enc_a = (enc_a * HE.encodeFrac(0.1)) - HE.encodeFrac(0.3)
# print(HE.decryptFrac(enc_a))

# print("First square")
# enc_a = HE.power(enc_a, 2)
# print(HE.decryptFrac(enc_a))

# print("Second conv")
# enc_a = (enc_a * HE.encodeFrac(0.2)) - HE.encodeFrac(0.1)
# print(HE.decryptFrac(enc_a))

# print("Second square")
# enc_a = HE.power(enc_a, 2)
# print(HE.decryptFrac(enc_a))

# print("First linear")
# enc_a = (enc_a * HE.encodeFrac(0.1)) + (enc_a * HE.encodeFrac(0.05)) + (enc_a * HE.encodeFrac(0.3)) + HE.encodeFrac(0.1)
# print(HE.decryptFrac(enc_a))

# print("Second linear")
# enc_a = (enc_a * HE.encodeFrac(-0.2)) + (enc_a * HE.encodeFrac(0.1)) + (enc_a * HE.encodeFrac(0.1)) + HE.encodeFrac(-0.4)
# print(HE.decryptFrac(enc_a))

# print("Third linear")
# enc_a = (enc_a * HE.encodeFrac(0.3)) + (enc_a * HE.encodeFrac(-0.2)) + (enc_a * HE.encodeFrac(0.3)) + HE.encodeFrac(0.2)
# print(HE.decryptFrac(enc_a))

# Models

In [5]:
class Square(torch.nn.Module):
    def __init__(self):
        super().__init__()
 
    def forward(self, t):
        return torch.pow(t, 2)

class Cube(torch.nn.Module):
    def __init__(self):
        super().__init__()
 
    def forward(self, t):
        return torch.pow(t, 3)
    
class Printer(torch.nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, t):
        # print(t)
        print(t.shape)
        return t


class PINPOINT_1CONV(nn.Module):
    def __init__(self, input_size, output_horizon):
        super(PINPOINT_1CONV, self).__init__()

        n_kernels_1 = 32
        kernel_size_1 = 3
        out_conv_1 = n_kernels_1 * (input_size - kernel_size_1 + 1)

        self.main = nn.Sequential(           
            nn.Conv1d(in_channels=1, out_channels=n_kernels_1, kernel_size=kernel_size_1),
            Square(),
            nn.Flatten(),      
            
            nn.Linear(out_conv_1, int(out_conv_1/2)), #use without avgpool
            # nn.Linear(int(out_conv_1/2), output_horizon)   
            nn.Linear(int(out_conv_1/2), int(out_conv_1/4)),
            nn.Linear(int(out_conv_1/4), output_horizon)   
        )

    def forward(self, x):
        for l in self.main:
            try:
                print(x[0][0][0])
            except Exception:
                print(x[0][0])
            x = l(x)
        print(x[0][0])
        return x
    
    def __str__(self):
        return "PINPOINT_1CONV"

    
class PINPOINT_2CONV(nn.Module):
    def __init__(self, input_size, output_horizon):
        super(PINPOINT_2CONV, self).__init__()
        
        n_kernels_1 = 16
        n_kernels_2 = 32
        kernel_size_1 = 5
        kernel_size_2 = 3
        
        out_conv_1 = input_size - kernel_size_1 + 1
        out_conv_2 = n_kernels_2 * (out_conv_1 - kernel_size_2 + 1)

        self.main = nn.Sequential(           
            nn.Conv1d(in_channels=1, out_channels=n_kernels_1, kernel_size=kernel_size_1),
            Square(),
            nn.Conv1d(in_channels=n_kernels_1, out_channels=n_kernels_2, kernel_size=kernel_size_2),
            Square(),
            nn.Flatten(),      
            
            nn.Linear(out_conv_2, int(out_conv_2/2)), #use without avgpool
            # nn.Linear(int(out_conv_2/4), output_horizon)   
            nn.Linear(int(out_conv_2/2), int(out_conv_2/4)),
            nn.Linear(int(out_conv_2/4), output_horizon)   
        )

    def forward(self, x):
        print(x)
        for l in self.main:
            try:
                print(x)
            except Exception:
                print(x[0][0])
            x = l(x)
        print(x[0][0])
        return x
    
        # out = self.main(x)
        # return out
    
    def __str__(self):
        return "PINPOINT_2CONV"

In [199]:
experiment_name = "Milk"
seq_length = 12
forecast_horizon = 6
model_class = "PINPOINT_2CONV"

In [200]:
model = torch.load(f"{working_directory}/Experiments/models/{experiment_name}_{forecast_horizon}_{model_class}.pt")

In [201]:
model

PINPOINT_2CONV(
  (main): Sequential(
    (0): Conv1d(1, 16, kernel_size=(5,), stride=(1,))
    (1): Square()
    (2): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
    (3): Square()
    (4): Flatten(start_dim=1, end_dim=-1)
    (5): Linear(in_features=192, out_features=96, bias=True)
    (6): Linear(in_features=96, out_features=48, bias=True)
    (7): Linear(in_features=48, out_features=6, bias=True)
  )
)

# Dataset

In [202]:
milk_production = milk_production = pd.read_csv(f"{working_directory}/data/monthly-milk-production.csv", parse_dates=["Month"], index_col="Month")
milk_production = milk_production.loc[:, 'Production']
milk_production.index.freq = 'MS'
entire_ts = milk_production
train = milk_production.loc[:pd.Timestamp("1974-01-01")]
validation_length = int(0.05 * len(train))
validation = entire_ts.loc[train.index[-1] + entire_ts.index.freq:train.index[-1] + validation_length * entire_ts.index.freq]
test = entire_ts.loc[validation.index[-1] + entire_ts.index.freq:]
plot_name = "Monthly milk production"
yaxis_name = "Production"

train = train.append(validation)

print(train)
print(test)

Month
1962-01-01    589
1962-02-01    561
1962-03-01    640
1962-04-01    656
1962-05-01    727
             ... 
1974-04-01    902
1974-05-01    969
1974-06-01    947
1974-07-01    908
1974-08-01    867
Freq: MS, Name: Production, Length: 152, dtype: int64
Month
1974-09-01    815
1974-10-01    812
1974-11-01    773
1974-12-01    813
1975-01-01    834
1975-02-01    782
1975-03-01    892
1975-04-01    903
1975-05-01    966
1975-06-01    937
1975-07-01    896
1975-08-01    858
1975-09-01    817
1975-10-01    827
1975-11-01    797
1975-12-01    843
Freq: MS, Name: Production, dtype: int64


# Expected outputs

In [203]:
expected_output = []

scaler = MinMaxScaler(feature_range=(-1, 1))
_ = scaler.fit_transform(train.values.reshape(-1, 1))

_train = train.copy()
_test = test.copy()

forecast = np.array([])

for i in range(0, int(len(_test) / forecast_horizon) + 1):
    if i > 0: 
        break
    model.eval()

    inputs = _train.values.reshape(len(_train), 1)

    inputs_normalized = scaler.transform(inputs)
    inputs_normalized = torch.FloatTensor(inputs_normalized[-seq_length:]).to(device)

    predict = model(inputs_normalized.reshape(1, 1, seq_length))
    predict = scaler.inverse_transform(predict.cpu().detach().numpy())
    forecast = np.append(forecast, predict)

    for j in range(0, forecast_horizon):
        if len(_test) > 0:
            _train[_train.index[-1] + train.index.freq] = _test.iloc[0]
            _test = _test.iloc[1:]

# expected_output = pd.Series(data=forecast[:len(test)], index=test.index)
expected_output = pd.Series(data=forecast[:len(test)])

tensor([[[ 0.1106,  0.1442, -0.0048,  0.1971,  0.3221,  0.0817,  0.6154,
           0.6779,  1.0000,  0.8942,  0.7067,  0.5096]]], device='cuda:0')
tensor([[[ 0.1106,  0.1442, -0.0048,  0.1971,  0.3221,  0.0817,  0.6154,
           0.6779,  1.0000,  0.8942,  0.7067,  0.5096]]], device='cuda:0')
tensor([[[-0.1528, -0.1989,  0.1217, -0.0887,  0.2874,  0.0773,  0.1500,
           0.0182],
         [ 0.2772,  0.4068,  0.0712,  0.2011,  0.1136,  0.0255,  0.0407,
          -0.0209],
         [-0.0813, -0.0741, -0.0850,  0.1376,  0.1174,  0.4252,  0.5376,
           0.6240],
         [ 0.3105,  0.3048,  0.4863,  0.5388,  0.8507,  0.8372,  0.9536,
           0.8414],
         [ 0.6594,  0.5616,  0.6862,  0.8686,  0.9524,  0.9204,  1.0105,
           0.9946],
         [ 0.4080,  0.4732,  0.5711,  0.3409,  0.5546,  0.4636,  0.3316,
           0.1734],
         [-0.3063, -0.0998, -0.2748, -0.4743, -0.4366, -0.3012, -0.3969,
          -0.4545],
         [ 0.6177,  0.5862,  0.8820,  0.8152,  1.0037

In [204]:
expected_output

0    812.044067
1    823.361755
2    764.408569
3    813.593750
4    839.368958
5    790.813354
dtype: float64

In [205]:
# print(experiment_name)
# print(f"MAE of model {model}, forecast horizon: {forecast_horizon}: {round(mean_absolute_error(test, expected_output), 2)}")

In [281]:
class ConvolutionalLayer:
    def __init__(self, weights, stride=(1, 1), padding=(0, 0), bias=None):
        self.weights = weights
        self.stride = stride
        self.padding = padding
        self.bias = bias
        if bias is not None:
            self.bias = bias

    def __call__(self, t):
        # t = apply_padding(t, self.padding)
        result = np.array([[np.sum([convolute1d(ts_layer, filter_layer, self.stride)
                                    for ts_layer, filter_layer in zip(_ts, _filter)], axis=0)
                             for _filter in self.weights] 
                           for _ts in t])

        if self.bias is not None:
            return np.array([[layer + bias.item() for layer, bias in zip(_ts, self.bias)] for _ts in result])
        else:
            return result


def convolute1d(ts, filter_matrix, stride):
    x_d = len(ts)
    x_f = len(filter_matrix)

    x_stride = stride[0]

    x_o = ((x_d - x_f) // x_stride) + 1

    def get_submatrix(matrix, x):
        index_column = x * x_stride
        return matrix[index_column: index_column + x_f]

    return np.array(
        [np.sum(get_submatrix(ts, x) * filter_matrix) for x in range(0, x_o)])

In [282]:
ts = np.array( [1, 2, 3, 4, 5, 6, 7, 8, 9] )
filter_matrix = np.array( [1, 2, 3] )
print(convolute1d(ts, filter_matrix, [1]))

[14 20 26 32 38 44 50]


In [283]:
model.eval()

PINPOINT_2CONV(
  (main): Sequential(
    (0): Conv1d(1, 16, kernel_size=(5,), stride=(1,))
    (1): Square()
    (2): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
    (3): Square()
    (4): Flatten(start_dim=1, end_dim=-1)
    (5): Linear(in_features=192, out_features=96, bias=True)
    (6): Linear(in_features=96, out_features=48, bias=True)
    (7): Linear(in_features=48, out_features=6, bias=True)
  )
)

In [284]:
torch_c1 = model.main[0].cpu()
torch_c2 = model.main[2].cpu()
c1 = ConvolutionalLayer(torch_c1.weight.detach().numpy(), torch_c1.stride, torch_c1.padding, torch_c1.bias)
c2 = ConvolutionalLayer(torch_c2.weight.detach().numpy(), torch_c2.stride, torch_c2.padding, torch_c2.bias)

In [285]:
inputs = _train.values.reshape(len(_train), 1)

inputs_normalized = scaler.transform(inputs)
inputs_normalized = torch.FloatTensor(inputs_normalized[-seq_length:]).to(device)
inputs_normalized = inputs_normalized.reshape(1, 1, seq_length)

In [286]:
inp = inputs_normalized.cpu().numpy()

In [287]:
torch_out_1 = torch_c1(inputs_normalized.cpu())
torch_out_2 = torch_c2(torch_out_1)

pycrcnn_out_1 = c1(inp)
pycrcnn_out_2 = c2(pycrcnn_out_1)

In [289]:
pycrcnn_out_2[0][0]

array([-0.32948136, -0.13188641, -0.10636571, -0.0558827 , -0.07941146,
       -0.09219632], dtype=float32)

In [290]:
torch_out_2[0][0]

tensor([-0.3295, -0.1319, -0.1064, -0.0559, -0.0794, -0.0922],
       grad_fn=<SelectBackward0>)

In [275]:
print(torch_out_1.shape)
print(pycrcnn_out_1.shape)

torch.Size([1, 16, 8])
(1, 16, 8)


In [276]:
print(torch_out_2.shape)
print(pycrcnn_out_2.shape)

torch.Size([1, 32, 6])
(1, 32, 6)


In [277]:
np.allclose(torch_out_1.detach().numpy(), pycrcnn_out_1)

True

In [278]:
np.allclose(torch_out_2.detach().numpy(), pycrcnn_out_2)

False

In [279]:
torch_out_2.detach().numpy() - pycrcnn_out_2

array([[[-2.9802322e-08,  1.4901161e-08,  4.4703484e-08,  0.0000000e+00,
          7.4505806e-09,  2.9802322e-08],
        [-5.9604645e-08, -2.9802322e-08,  2.9802322e-08,  0.0000000e+00,
          0.0000000e+00, -1.4901161e-08],
        [ 7.4505806e-08,  0.0000000e+00,  0.0000000e+00, -2.9802322e-08,
         -2.9802322e-08,  1.4901161e-08],
        [ 5.9604645e-08, -2.9802322e-08,  0.0000000e+00,  0.0000000e+00,
         -2.9802322e-08, -1.8626451e-08],
        [ 0.0000000e+00,  2.9802322e-08, -8.9406967e-08,  4.4703484e-08,
          0.0000000e+00, -5.9604645e-08],
        [-2.9802322e-08,  1.8626451e-08,  1.4901161e-08, -7.4505806e-09,
          7.4505806e-09, -1.4901161e-08],
        [ 0.0000000e+00,  2.9802322e-08, -2.9802322e-08,  0.0000000e+00,
          2.9802322e-08, -1.4901161e-08],
        [ 1.1920929e-07,  1.1920929e-07,  0.0000000e+00,  5.9604645e-08,
          0.0000000e+00, -5.9604645e-08],
        [-4.4703484e-08,  1.4901161e-08,  2.9802322e-08, -2.9802322e-08,
       

In [280]:
pycrcnn_out_2

array([[[-3.29481363e-01, -1.31886408e-01, -1.06365710e-01,
         -5.58826961e-02, -7.94114619e-02, -9.21963155e-02],
        [-2.43459761e-01, -3.51462454e-01, -4.29031104e-01,
         -3.42738330e-01, -2.42686629e-01, -1.65710464e-01],
        [ 1.37510374e-01,  2.10252553e-01,  2.79890716e-01,
          2.91717201e-01,  2.30780423e-01,  2.07163870e-01],
        [-3.07030082e-01, -3.85308117e-01, -3.32821250e-01,
         -2.69986808e-01, -1.55314639e-01, -4.56414111e-02],
        [ 1.81319341e-01,  2.34996319e-01,  2.41125494e-01,
          1.85159400e-01,  1.19449630e-01,  9.38674212e-02],
        [-1.49477601e-01, -2.71312036e-02, -4.92407046e-02,
         -4.06488366e-02, -2.71976143e-02, -1.08691305e-01],
        [-2.52759367e-01, -3.25511247e-01, -3.85629773e-01,
         -2.96744049e-01, -1.78671360e-01, -1.37597203e-01],
        [ 7.36791074e-01,  8.58288705e-01,  8.84172738e-01,
          8.51966918e-01,  7.39866555e-01,  5.94550312e-01],
        [ 1.03070833e-01,  1.946

In [167]:
inp.shape

(1, 1, 12)

In [168]:
sample_inp = np.array([ [ [ 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0 ] ] ])

In [169]:
sample_inp.shape

(1, 1, 9)

In [170]:
conv1 = nn.Conv1d(in_channels=1, out_channels=2, kernel_size=3, bias=False)

conv1.weight = nn.Parameter(torch.Tensor(sample_weight_1))

In [171]:
conv2 = nn.Conv1d(in_channels=2, out_channels=3, kernel_size=3, bias=False)

conv2.weight = nn.Parameter(torch.Tensor(sample_weight_c2))

In [172]:
sample_weight_1 = np.array([ [ [ 1.0, 2.0, 3.0 ] ] , [[ 2.0, 3.0, 4.0 ]] ])
sample_c1 = ConvolutionalLayer(sample_weight_1, conv1.stride, conv1.padding)

In [173]:
sample_weight_c2 = np.array([ [ [ 1.0, 2.0, 3.0 ], [2.0, 3.0, 4.0] ], 
                              [ [ 5.0, 6.0, 7.0 ], [8.0, 9.0, 10.0] ], 
                              [ [ 11.0, 12.0, 13.0 ], [14.0, 15.0, 16.0] ] ])
sample_c2 = ConvolutionalLayer(sample_weight_c2, conv2.stride, conv1.padding)

In [174]:
a = conv1(torch.Tensor(sample_inp))

In [175]:
a

tensor([[[14., 20., 26., 32., 38., 44., 50.],
         [20., 29., 38., 47., 56., 65., 74.]]], grad_fn=<ConvolutionBackward0>)

In [176]:
conv2(a)

tensor([[[ 411.,  528.,  645.,  762.,  879.],
         [1173., 1524., 1875., 2226., 2577.],
         [2055., 2676., 3297., 3918., 4539.]]], grad_fn=<ConvolutionBackward0>)

In [177]:
b = sample_c1(sample_inp)

In [178]:
b

array([[[14., 20., 26., 32., 38., 44., 50.],
        [20., 29., 38., 47., 56., 65., 74.]]])

In [179]:
sample_c2(b)

array([[[ 411.,  528.,  645.,  762.,  879.],
        [1173., 1524., 1875., 2226., 2577.],
        [2055., 2676., 3297., 3918., 4539.]]])

In [295]:
class LinearLayer:
    def __init__(self, weights, bias=None):
        self.weights = weights
        self.bias = bias
        if bias is not None:
            self.bias = bias

    def __call__(self, t):
        result = np.array([[np.sum(ts * row) for row in self.weights] for ts in t])
        if self.bias is not None:
            result = np.array([row + self.bias for row in result])
        return result

In [312]:
lin1 = model.main[5].cpu()
lin2 = model.main[6].cpu()
lin3 = model.main[7].cpu()

pycrcnn_lin1 = LinearLayer(lin1.weight.detach().numpy(), lin1.bias.detach().numpy())
pycrcnn_lin2 = LinearLayer(lin2.weight.detach().numpy(), lin2.bias.detach().numpy())
pycrcnn_lin3 = LinearLayer(lin3.weight.detach().numpy(), lin3.bias.detach().numpy())
# c1 = ConvolutionalLayer(torch_c1.weight.detach().numpy(), torch_c1.stride, torch_c1.padding, torch_c1.bias)
# c2 = ConvolutionalLayer(torch_c2.weight.detach().numpy(), torch_c2.stride, torch_c2.padding, torch_c2.bias)

In [313]:
inp = np.ones(192).reshape(1, 192)

In [314]:
out1 = lin1(torch.Tensor(inp))
out2 = lin2(out1)
out3 = lin3(out2)

In [315]:
out3

tensor([[-2.4264, -1.6281, -2.2950, -1.8667, -1.8219, -2.1086]],
       grad_fn=<AddmmBackward0>)

In [316]:
pycrcnn_out1 = pycrcnn_lin1(inp)
pycrcnn_out2 = pycrcnn_lin2(pycrcnn_out1)
pycrcnn_out3 = pycrcnn_lin3(pycrcnn_out2)

In [317]:
pycrcnn_out3

array([[-2.42642315, -1.62807017, -2.29497676, -1.86671326, -1.82189733,
        -2.10862728]])